In [231]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import preprocessing

# Foreclosure Data

In [265]:
file = pd.read_csv('foreclosure_data.csv')
original = file.copy(deep=True)


In [257]:
colvals = file.columns.tolist()
colvals

['loanID',
 'origChannel',
 'sellerName',
 'origIntRate',
 'orgUPB',
 'orgLoanTerm',
 'orginationDate',
 'firstPaymentDate',
 'orgLTV',
 'orgCombinedLTV',
 'numBorrowers',
 'orgDebttoIncomeRatio',
 'borrowerCreditScoreAtOrigination',
 'firstTimeHomeBuyer',
 'loanPurpose',
 'propType',
 'numUnits',
 'occType',
 'propertyState',
 'zipCodeShort',
 'priMortgageInsurancePercent',
 'prodType',
 'coBorrowerCreditScoreAtOrigination',
 'mortInsType',
 'reloMortIndicator',
 'foreclosed']

In [234]:
file.describe

<bound method NDFrame.describe of             loanID origChannel                                 sellerName  \
0     100007365142           R                    JPMORGAN CHASE BANK  NA   
1     100007386460           B                    JPMORGAN CHASE BANK  NA   
2     100023274028           R  FIRST TENNESSEE BANK NATIONAL ASSOCIATION   
3     100026859568           C  FIRST TENNESSEE BANK NATIONAL ASSOCIATION   
4     100028993931           R                     WELLS FARGO BANK  N.A.   
5     100035719682           R                   JP MORGAN CHASE BANK  NA   
6     100048724218           B                    JPMORGAN CHASE BANK  NA   
7     100057706656           B                         FLAGSTAR BANK  FSB   
8     100079064875           R                         GMAC MORTGAGE  LLC   
9     100088638564           R                  USAA FEDERAL SAVINGS BANK   
10    100094030068           C                         CITIMORTGAGE  INC.   
11    100100228738           R            

In [235]:
todrop=[]
indtokeep = [3,5,9,10,11,12,19,20,26]
i=1
for x in colvals:
    if i not in indtokeep:
        todrop.append(x)
    i += 1

In [236]:
file= file.drop(columns=todrop)
file=file.dropna(axis='rows')
file.columns.values

array(['sellerName', 'orgUPB', 'orgLTV', 'orgCombinedLTV', 'numBorrowers',
       'orgDebttoIncomeRatio', 'propertyState', 'zipCodeShort',
       'foreclosed'], dtype=object)

In [237]:
file.info()
org_file=file
file.describe

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 367 to 4705
Data columns (total 9 columns):
sellerName              17 non-null object
orgUPB                  17 non-null int64
orgLTV                  17 non-null int64
orgCombinedLTV          17 non-null float64
numBorrowers            17 non-null float64
orgDebttoIncomeRatio    17 non-null float64
propertyState           17 non-null object
zipCodeShort            17 non-null int64
foreclosed              17 non-null object
dtypes: float64(3), int64(3), object(3)
memory usage: 1.3+ KB


<bound method NDFrame.describe of                    sellerName  orgUPB  orgLTV  orgCombinedLTV  numBorrowers  \
367   JPMORGAN CHASE BANK  NA  187000      80            90.0           2.0   
384   JPMORGAN CHASE BANK  NA  150000      80            95.0           1.0   
526   JPMORGAN CHASE BANK  NA  110000      80            95.0           1.0   
807              REGIONS BANK  118000      80            90.0           2.0   
872              AMTRUST BANK  200000      68            70.0           2.0   
901   JPMORGAN CHASE BANK  NA  140000      80            95.0           1.0   
1735             AMTRUST BANK   75000      64            65.0           2.0   
1901                    OTHER   98000      80            95.0           1.0   
2320  JPMORGAN CHASE BANK  NA  253000      79            89.0           2.0   
2405  JPMORGAN CHASE BANK  NA  153000      80            90.0           2.0   
2512                    OTHER   90000      44            54.0           2.0   
2710  JPMORGAN CHA

In [238]:
# more pre processing
le = preprocessing.LabelEncoder()
seller = le.fit(file['sellerName'])
list(seller.classes_)
seller.transform(file['sellerName'])
file['seller'] = seller.transform(file['sellerName'])
state = le.fit(file['propertyState'])
list(state.classes_)
file['state'] = state.transform(file['propertyState'])
file = file.drop(columns=['sellerName','propertyState'])
file.columns.tolist()
file.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17 entries, 367 to 4705
Data columns (total 9 columns):
orgUPB                  17 non-null int64
orgLTV                  17 non-null int64
orgCombinedLTV          17 non-null float64
numBorrowers            17 non-null float64
orgDebttoIncomeRatio    17 non-null float64
zipCodeShort            17 non-null int64
foreclosed              17 non-null object
seller                  17 non-null int64
state                   17 non-null int64
dtypes: float64(3), int64(5), object(1)
memory usage: 1.3+ KB


In [239]:
# scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
to_scale = ['orgUPB','orgLTV','orgCombinedLTV','numBorrowers','orgDebttoIncomeRatio']
file[['orgUPB','orgLTV','orgCombinedLTV','numBorrowers','orgDebttoIncomeRatio']] = scaler.fit_transform(file[['orgUPB','orgLTV','orgCombinedLTV','numBorrowers','orgDebttoIncomeRatio']])#file.describe

#scaler.fit_transform(file[['orgUPB']])
#scaler.transform([[75000]])
#scaler.fit_transform(file[['orgUPB']])
#scaler.fit_transform([[130000]])

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [240]:
# scaler fn
def scale(X,orig):
    maxv = orig.max()
    minv = orig.min()
    X_std = (X - minv) / (maxv - minv)
    #X_scaled = X_std * (maxv - minv) + minv
    return [X_std]

In [241]:
# random shuffle
file_shuffled = shuffle(file, random_state=0)
x = file_shuffled.drop(columns=['foreclosed'])
y = file_shuffled['foreclosed']

In [242]:
# divide into training and test
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, random_state=42)

In [243]:
# fit model
clf = tree.DecisionTreeClassifier() #decision tree gives best results as per WEKA
clf = clf.fit(X_train, y_train)

In [244]:
# test accuracy
from sklearn.metrics import accuracy_score
y_predict = clf.predict(X_test)
accuracy_score(y_test, y_predict)

1.0

In [245]:
yt = y_test.tolist()
yp = y_predict.tolist()
type(yt)

list

In [246]:
for v in range(len(y_test)):
    print(f'actual: {yt[v]}, p: {yp[v]}')

actual: no, p: no
actual: no, p: no
actual: no, p: no
actual: no, p: no
actual: no, p: no
actual: no, p: no


In [247]:
def foreclosure_test_model(orgUPB=75000,orgLTV=79,orgCLTV=54,numB=2,dtir=31.0,zipc=173,seller_='JPMORGAN CHASE BANK  NA',state_='TX'):
    dic = {'orgUPB':[orgUPB], 'orgLTV':[orgLTV], 'orgCombinedLTV':[orgCLTV], 'numBorrowers':[numB], 'orgDebttoIncomeRatio':[dtir],'zipCodeShort':[zipc],'seller':[seller_],'state':[state_]}
    df = pd.DataFrame.from_dict(dic)
    seller = le.fit(org_file['sellerName'])
    df['seller']=seller.transform(df['seller'])
    state = le.fit(org_file['propertyState'])
    df['state']=state.transform(df['state'])
    df['orgUPB']= scale(df['orgUPB'][0], org_file['orgUPB'])
    df['orgLTV']= scale(df['orgLTV'][0], org_file['orgLTV'])
    df['orgCombinedLTV']= scale(df['orgCombinedLTV'][0], org_file['orgCombinedLTV'])
    df['numBorrowers']= scale(df['numBorrowers'][0], org_file['numBorrowers'])
    df['orgDebttoIncomeRatio']= scale(df['orgDebttoIncomeRatio'][0], org_file['orgDebttoIncomeRatio'])
    ypred = clf.predict(df)
    return ypred

In [248]:
# test
'''
orgUPB=75000
orgLTV=79
orgCLTV=54
numB=2
dtir=31.0
zipc=173
seller_='JPMORGAN CHASE BANK  NA'
state_='TX'
dic = {'orgUPB':[orgUPB], 'orgLTV':[orgLTV], 'orgCombinedLTV':[orgCLTV], 'numBorrowers':[numB], 'orgDebttoIncomeRatio':[dtir],'zipCodeShort':[zipc],'seller':[seller_],'state':[state_]}
df = pd.DataFrame.from_dict(dic)
seller = le.fit(org_file['sellerName'])
df['seller']=seller.transform(df['seller'])
state = le.fit(org_file['propertyState'])
df['state']=state.transform(df['state'])
df['orgUPB']= scale(df['orgUPB'][0], org_file['orgUPB'])
df['orgLTV']= scale(df['orgLTV'][0], org_file['orgLTV'])
df['orgCombinedLTV']= scale(df['orgCombinedLTV'][0], org_file['orgCombinedLTV'])
df['numBorrowers']= scale(df['numBorrowers'][0], org_file['numBorrowers'])
df['orgDebttoIncomeRatio']= scale(df['orgDebttoIncomeRatio'][0], org_file['orgDebttoIncomeRatio'])
df
'''

"\norgUPB=75000\norgLTV=79\norgCLTV=54\nnumB=2\ndtir=31.0\nzipc=173\nseller_='JPMORGAN CHASE BANK  NA'\nstate_='TX'\ndic = {'orgUPB':[orgUPB], 'orgLTV':[orgLTV], 'orgCombinedLTV':[orgCLTV], 'numBorrowers':[numB], 'orgDebttoIncomeRatio':[dtir],'zipCodeShort':[zipc],'seller':[seller_],'state':[state_]}\ndf = pd.DataFrame.from_dict(dic)\nseller = le.fit(org_file['sellerName'])\ndf['seller']=seller.transform(df['seller'])\nstate = le.fit(org_file['propertyState'])\ndf['state']=state.transform(df['state'])\ndf['orgUPB']= scale(df['orgUPB'][0], org_file['orgUPB'])\ndf['orgLTV']= scale(df['orgLTV'][0], org_file['orgLTV'])\ndf['orgCombinedLTV']= scale(df['orgCombinedLTV'][0], org_file['orgCombinedLTV'])\ndf['numBorrowers']= scale(df['numBorrowers'][0], org_file['numBorrowers'])\ndf['orgDebttoIncomeRatio']= scale(df['orgDebttoIncomeRatio'][0], org_file['orgDebttoIncomeRatio'])\ndf\n"

In [249]:
# delinquency

In [250]:
# zero balance code

# Bokeh Structure 

In [270]:
import bokeh
from bokeh.models.glyphs import Text
from bokeh.io import output_notebook, show
#from bokeh.plotting import output_server
from bokeh.layouts import widgetbox
from bokeh.models.widgets import Button, RadioButtonGroup, Select, Slider, TextInput

In [271]:
original['sellerName'].unique()
original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4892 entries, 0 to 4891
Data columns (total 26 columns):
loanID                                4892 non-null int64
origChannel                           4892 non-null object
sellerName                            4892 non-null object
origIntRate                           4892 non-null float64
orgUPB                                4892 non-null int64
orgLoanTerm                           4892 non-null int64
orginationDate                        4892 non-null object
firstPaymentDate                      4892 non-null object
orgLTV                                4892 non-null int64
orgCombinedLTV                        18 non-null float64
numBorrowers                          4886 non-null float64
orgDebttoIncomeRatio                  4697 non-null float64
borrowerCreditScoreAtOrigination      4759 non-null float64
firstTimeHomeBuyer                    4892 non-null object
loanPurpose                           4892 non-null object
propType 

In [272]:
select1 = Select(title="Seller", value="foo", options=original['sellerName'].tolist())
select2 = Select(title="State", value="foo", options=original['propertyState'].tolist())
#heading = Text('Foreclosure Prediction:')
button_1 = Button(label="Foreclosure Prediction:")
upb = TextInput(
    value='75000',
    title='origUPB',
)
ltv = TextInput(    value='79',    title='orig ltv',)
cltv = TextInput(    value='54',    title='orig cltv',)
numb = TextInput(    value='2',    title='num of borrowers',)
dtir = TextInput(    value='31.0',    title='dti ratio',)
show(widgetbox(button_1, select1, select2, upb,width=300))
output_notebook

<function bokeh.io.output.output_notebook>